<a href="https://colab.research.google.com/github/Linsen-gao-457/Build_Forrest_LLM/blob/main/Step_2_Coding_Attention_Mechanisms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simplifies self-attention

**Step1**: compute unnormalized attention scores $ω$

In [ ]:
from importlib.metadata import version
print(f"torch version:{version('torch')}")

torch version:2.6.0+cu124


In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

In [ ]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(query, x_i)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
res = 0
for idx, element in enumerate(inputs[0]):
  res += element * inputs[1][idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


**Step 2**: normalize the unnormalized attention scores using softmax

In [ ]:
attn_weights_2_tmp = attn_scores_2 / torch.sum(attn_scores_2)
print(f"Normalized attention weights: {attn_weights_2_tmp}")
print(f"Sum: {attn_weights_2_tmp.sum()}")

Normalized attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: 1.0000001192092896


In [ ]:
def softmax(x):
  return torch.exp(x) / torch.exp(x).sum()

attn_weights_2_naive = softmax(attn_scores_2)
print(f"Normalized attention weights: {attn_weights_2_naive}")
print(f"Sum: {attn_weights_2_naive.sum()}")

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(f"Normalized attention weights: {attn_weights_2}")
print(f"Sum: {attn_weights_2.sum()}")

Normalized attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0
Normalized attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


**Step3** compute the context vector $z^{(2)}$ by multiplying the embedded input tokens, $x^{(i)}$ with the attention weights and sum the resulting vectors:

In [ ]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [ ]:
attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
  for j, x_j in enumerate(inputs):
    attn_scores[i,j] = torch.dot(x_i, x_j)

print(attn_scores)



tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [ ]:
row_2_sum = sum(attn_weights[2])
print(row_2_sum)
print("All row sums:", attn_weights.sum(dim=-1))

tensor(1.)
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [ ]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


#Implementing self-attention with trainable weights

In [ ]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [ ]:
keys = inputs @ W_key
values = inputs @ W_value
print(f"keys.shape{keys.shape}")
print(f"values,shape{values.shape}")

keys.shapetorch.Size([6, 2])
values,shapetorch.Size([6, 2])


In [ ]:
keys_2 = keys[1]
attn_scores_22 = query_2.dot(keys_2)
print(attn_scores_22)

tensor(1.8524)


In [ ]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [ ]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / (d_k**0.5), dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [ ]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


Implementing a compact self-attention python class

In [ ]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
  def __init__(self, d_in, d_out):
    super().__init__()
    self.W_query = nn.Parameter(torch.rand(d_in, d_out))
    self.W_key = nn.Parameter(torch.rand(d_in, d_out))
    self.W_value = nn.Parameter(torch.rand(d_in, d_out))

  def forward(self,x):
    keys = x @ self.W_key
    queries = x @ self.W_query
    values = x @ self.W_value
    attn_scores = queries @ keys.T
    atten_weights = torch.softmax(attn_scores / (d_in**0.5), dim=-1)
    context_vecs = atten_weights @ values
    return context_vecs

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
context_vecs = sa_v1(inputs)
print(context_vecs)

tensor([[0.2961, 0.7970],
        [0.3016, 0.8104],
        [0.3013, 0.8098],
        [0.2921, 0.7873],
        [0.2904, 0.7833],
        [0.2956, 0.7959]], grad_fn=<MmBackward0>)


In [ ]:
class SelfAttention_v2 (nn.Module):

  def __init__(self, d_in, d_out, qkv_bias=False):
    super().__init__()
    self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
    self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

  def forward(self, x):
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores / (d_in**0.5), dim=-1)
    context_vecs = attn_weights @ values
    return context_vecs

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
context_vecs = sa_v2(inputs)
print(context_vecs)

tensor([[-0.0745,  0.0704],
        [-0.0752,  0.0697],
        [-0.0752,  0.0696],
        [-0.0762,  0.0681],
        [-0.0765,  0.0677],
        [-0.0757,  0.0688]], grad_fn=<MmBackward0>)


In [ ]:
qeuries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = qeuries @ keys.T
attn_weights = torch.softmax(attn_scores / (d_in**0.5), dim=-1)
print(attn_weights)

tensor([[0.1873, 0.1651, 0.1655, 0.1571, 0.1712, 0.1538],
        [0.1968, 0.1662, 0.1664, 0.1527, 0.1666, 0.1512],
        [0.1964, 0.1662, 0.1665, 0.1529, 0.1666, 0.1514],
        [0.1831, 0.1667, 0.1668, 0.1589, 0.1662, 0.1583],
        [0.1800, 0.1669, 0.1669, 0.1603, 0.1660, 0.1600],
        [0.1883, 0.1665, 0.1667, 0.1565, 0.1667, 0.1554]],
       grad_fn=<SoftmaxBackward0>)


In [ ]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)
masked_simple = attn_weights * mask_simple
print(masked_simple)
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.1873, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1968, 0.1662, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1964, 0.1662, 0.1665, 0.0000, 0.0000, 0.0000],
        [0.1831, 0.1667, 0.1668, 0.1589, 0.0000, 0.0000],
        [0.1800, 0.1669, 0.1669, 0.1603, 0.1660, 0.0000],
        [0.1883, 0.1665, 0.1667, 0.1565, 0.1667, 0.1554]],
       grad_fn=<MulBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5422, 0.4578, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3712, 0.3141, 0.3146, 0.0000, 0.0000, 0.0000],
        [0.2710, 0.2468, 0.2470, 0.2352, 0.0000, 0.0000],
        [0.2142, 0.1986, 0.1987, 0.1908, 0.1976, 0.0000],
        [0.1883, 0.1665, 0.1667, 0.1565, 0.1667, 0.1554]],
       grad_fn=<DivBackward0>)
